In [1]:
import time
import pandas as pd
import requests
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup, element

In [25]:
def main_fun(city, page_num):
    # Get the HTML from the page
    url = 'https://www.otodom.pl'
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')
    # search input
    key = city
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.get(url)
    time.sleep(2)
    #cookie click
    cookie_click = driver.find_element(By.XPATH, '//*[@id="onetrust-accept-btn-handler"]')
    cookie_click.click()
    time.sleep(2)
    #search bar click
    search_bar = driver.find_element(By.XPATH, '//*[@id="location"]/div[2]')
    search_bar.click()
    time.sleep(2)
    #search entry
    search_entry = driver.find_element(By.XPATH, '//*[@id="location-picker-input"]')
    search_entry.send_keys(key)
    time.sleep(2)
    #search select
    search_select = driver.find_element(By.XPATH, '//*[@id="__next"]/main/section/div/div/form/div/div[1]/div[3]/div/div[1]/div/div[2]/ul/li[1]/label[1]')
    search_select.click()
    time.sleep(2)
    #search button press
    search_button = driver.find_element(By.XPATH, '//*[@id="search-form-submit"]')
    search_button.click()
    time.sleep(5)
    # getting apartments data
    url = driver.current_url
    res = requests.get(url)
    time.sleep(5)
    # scroll to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    button = driver.find_element(By.XPATH, '//*[@id="__next"]/div[2]/main/div/div[2]/div[1]/div[4]/div/div/div/div/div/div[2]/div')
    button.click()
    time.sleep(3)
    # change the number of listings to be shown
    pages = driver.find_element(By.XPATH, '//*[@id="react-select-2-option-3"]')
    pages.click()
    time.sleep(5)
    df_otodom = pd.DataFrame()
    for i in range(0, page_num):
        if page_num == 1:
            # scroll to bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            df_otodom = pd.DataFrame()
            # getting apartments data
            url = driver.current_url
            res = requests.get(url)
            time.sleep(3)
            # scroll to bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            apartment = soup.find_all('li', {'class': 'css-iq9jxc e1n6ljqa1'})
            apt: element.Tag
            apartments = list()
            for apt in apartment:
                try:
                    apartments.append({
                        'name': apt.find('h3', {'class': 'css-1mmyqk7 e1n6ljqa6'}).text,
                        'city': key,
                        'price': apt.find('span', {'class': 'css-1ntk0hg ei6hyam1'}).text,
                        'rooms': apt.find('div', {'class': 'e1n6ljqa19 css-6vtodn ei6hyam0'}).find_all('span')[2].get_text(),
                        'area': (apt.find('div', {'class': 'e1n6ljqa19 css-6vtodn ei6hyam0'}).find_all('span')[3].get_text()),
                        'price_per_m2': apt.find('div', {'class': 'e1n6ljqa19 css-6vtodn ei6hyam0'}).find_all('span')[1].get_text(),
                        'link': url + apt.find('a', {'class': 'css-1up0y1q e1n6ljqa16'}).get('href')
                    })
                except:
                    pass
            df = pd.DataFrame(apartments)
            # df link column to list
            links = df['link'].tolist()
            good_links = list()
            for i in range(len(links)):
                url = links[i]
                prefix = 'https://www.otodom.pl'
                suffix = url.split('/oferta/')[-1]
                result = prefix + '/oferta/' + suffix
                good_links.append(result)

            # replace df link column with good_links
            df['link'] = good_links
            
            additions = list()
            for x in range(len(df['link'])):
                url = df['link'][x]
                res = requests.get(url)
                soup = BeautifulSoup(res.text, 'html.parser')
                try:
                    forma_wlasnosci = soup.find('div', {'aria-label': 'Forma własności'}).find_all('div')[3].text
                except:
                    forma_wlasnosci = None
                    
                try:
                    stan_wykonczenia = soup.find('div', {'aria-label': 'Stan wykończenia'}).find_all('div')[3].text
                except:
                    stan_wykonczenia = None
                    
                try:
                    pietro = soup.find('div', {'aria-label': 'Piętro'}).find_all('div')[3].text
                except:
                    pietro = None
                    
                try:
                    balkon = soup.find('div', {'aria-label': 'Balkon / ogród / taras'}).find_all('div')[3].text
                except:
                    balkon = None
                    
                try:
                    czynsz = soup.find('div', {'aria-label': 'Czynsz'}).find_all('div')[3].text
                except:
                    czynsz = None
                
                try:
                    parking = soup.find('div', {'aria-label': 'Miejsce parkingowe'}).find_all('div')[3].text
                except:
                    parking = None
                    
                try:
                    ogrzewanie = soup.find('div', {'aria-label': 'Ogrzewanie'}).find_all('div')[3].text
                except:
                    ogrzewanie = None
                
                additions.append({
                'forma_wlasnosci': forma_wlasnosci,
                'stan_wykonczenia': stan_wykonczenia,
                'pietro': pietro,
                'balkon': balkon,
                'czynsz': czynsz,
                'parking': parking,
                'ogrzewanie': ogrzewanie
                })
            df = df.join(pd.DataFrame(additions))
            df_otodom = df_otodom.append(df)
        else:
            # scroll to bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            # getting apartments data
            url = driver.current_url
            res = requests.get(url)
            time.sleep(3)
            # scroll to bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            apartment = soup.find_all('li', {'class': 'css-iq9jxc e1n6ljqa1'})
            apt: element.Tag
            apartments = list()
            for apt in apartment:
                try:
                    apartments.append({
                        'name': apt.find('h3', {'class': 'css-1mmyqk7 e1n6ljqa6'}).text,
                        'city': key,
                        'price': apt.find('span', {'class': 'css-1ntk0hg ei6hyam1'}).text,
                        'rooms': apt.find('div', {'class': 'e1n6ljqa19 css-6vtodn ei6hyam0'}).find_all('span')[2].get_text(),
                        'area': (apt.find('div', {'class': 'e1n6ljqa19 css-6vtodn ei6hyam0'}).find_all('span')[3].get_text()),
                        'price_per_m2': apt.find('div', {'class': 'e1n6ljqa19 css-6vtodn ei6hyam0'}).find_all('span')[1].get_text(),
                        'link': url + apt.find('a', {'class': 'css-1up0y1q e1n6ljqa16'}).get('href')
                    })
                except:
                    pass
            df = pd.DataFrame(apartments)
            
            links = df['link'].tolist()
            good_links = list()
            for i in range(len(links)):
                url = links[i]
                prefix = 'https://www.otodom.pl'
                suffix = url.split('/oferta/')[-1]
                result = prefix + '/oferta/' + suffix
                good_links.append(result)

            # replace df link column with good_links
            df['link'] = good_links
            
            additions = list()
            for x in range(len(df['link'])):
                url = df['link'][x]
                res = requests.get(url)
                soup = BeautifulSoup(res.text, 'html.parser')
                try:
                    forma_wlasnosci = soup.find('div', {'aria-label': 'Forma własności'}).find_all('div')[3].text
                except:
                    forma_wlasnosci = None
                    
                try:
                    stan_wykonczenia = soup.find('div', {'aria-label': 'Stan wykończenia'}).find_all('div')[3].text
                except:
                    stan_wykonczenia = None
                    
                try:
                    pietro = soup.find('div', {'aria-label': 'Piętro'}).find_all('div')[3].text
                except:
                    pietro = None
                    
                try:
                    balkon = soup.find('div', {'aria-label': 'Balkon / ogród / taras'}).find_all('div')[3].text
                except:
                    balkon = None
                    
                try:
                    czynsz = soup.find('div', {'aria-label': 'Czynsz'}).find_all('div')[3].text
                except:
                    czynsz = None
                
                try:
                    parking = soup.find('div', {'aria-label': 'Miejsce parkingowe'}).find_all('div')[3].text
                except:
                    parking = None
                    
                try:
                    ogrzewanie = soup.find('div', {'aria-label': 'Ogrzewanie'}).find_all('div')[3].text
                except:
                    ogrzewanie = None
                
                additions.append({
                'forma_wlasnosci': forma_wlasnosci,
                'stan_wykonczenia': stan_wykonczenia,
                'pietro': pietro,
                'balkon': balkon,
                'czynsz': czynsz,
                'parking': parking,
                'ogrzewanie': ogrzewanie
                })
            df = df.join(pd.DataFrame(additions))
            df_otodom = df_otodom.append(df)
            page_button = driver.find_element(By.XPATH, '//*[@id="__next"]/div[2]/main/div/div[2]/div[1]/div[4]/div/nav/button[6]')
            page_button.click()
            time.sleep(3)
            continue
    df_otodom.drop_duplicates(subset=['name', 'link'], inplace=True).reset_index(drop=True)
    return df_otodom

In [30]:
otodom_main = main_fun('Warszawa', 2)

C:\Users\48518\AppData\Local\Temp\ipykernel_9172\1412763198.py:231: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_otodom = df_otodom.append(df)
C:\Users\48518\AppData\Local\Temp\ipykernel_9172\1412763198.py:231: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_otodom = df_otodom.append(df)


In [32]:
otodom_main

,name,city,price,rooms,area,price_per_m2,link,forma_wlasnosci,stan_wykonczenia,pietro,balkon,czynsz,parking,ogrzewanie
0,"Mieszkanie 3 pok. z tarasem, balkon Skarbka z Gór",Warszawa,660 000 zł,3 pokoje,69.9 m²,9442 zł/m²,https://www.otodom.pl/oferta/mieszkanie-3-pok-...,pełna własność,do zamieszkania,parter/2,"balkon, taras",None,garaż/miejsce parkingowe,None
1,3 pokoje Jelonki dwustronne gotowe do zamieszk...,Warszawa,780 000 zł,3 pokoje,60 m²,13 000 zł/m²,https://www.otodom.pl/oferta/3-pokoje-jelonki-...,spółdzielcze własnościowe,do zamieszkania,3/12,balkon,570 zł,None,miejskie
2,"Ciekawe, dwupokojowe mieszkanie na MDM-ie.",Warszawa,1 495 000 zł,2 pokoje,65 m²,23 000 zł/m²,https://www.otodom.pl/oferta/ciekawe-dwupokojo...,pełna własność,do zamieszkania,3,balkon,702 zł,None,miejskie
3,4-pokojowe mieszkanie 80m2 + 2 loggie,Warszawa,Zapytaj o cenę,4 pokoje,80.12 m²,,https://www.otodom.pl/oferta/4-pokojowe-mieszk...,pełna własność,do wykończenia,1/3,None,None,garaż/miejsce parkingowe,None
4,3-pokojowe mieszkanie 59m2 + 2 ogródki,Warszawa,861 204 zł,3 pokoje,59.23 m²,14 540 zł/m²,https://www.otodom.pl/oferta/3-pokojowe-mieszk...,pełna własność,do wykończenia,parter/8,"taras, ogródek",None,garaż/miejsce parkingowe,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,"Komfortowe, 2 pokoje, Ursus",Warszawa,735 000 zł,2 pokoje,56.6 m²,12 986 zł/m²,https://www.otodom.pl/oferta/komfortowe-2-poko...,None,None,5/6,balkon,620 zł,garaż/miejsce parkingowe,miejskie
71,2 pokoje Stara Ochota z ogromnym tarasem na da...,Warszawa,552 000 zł,2 pokoje,40 m²,13 800 zł/m²,https://www.otodom.pl/oferta/2-pokoje-stara-oc...,None,do zamieszkania,1/5,"balkon, taras",400 zł,None,miejskie
72,Mieszkanie z duszą w sercu miasta | 300m do metra,Warszawa,1 020 000 zł,2 pokoje,50 m²,20 400 zł/m²,https://www.otodom.pl/oferta/mieszkanie-z-dusz...,pełna własność,do wykończenia,3/5,None,900 zł,None,miejskie
73,Wyjątkowo cicha lokalizacja | 8 minut Metro Mł...,Warszawa,560 000 zł,2 pokoje,38 m²,14 737 zł/m²,https://www.otodom.pl/oferta/wyjatkowo-cicha-l...,pełna własność,do zamieszkania,2/3,balkon,None,None,miejskie


WORK

In [20]:
def fun(df):
    # df link column to list
    links = df['link'].tolist()
    good_links = list()
    for i in range(len(links)):
        url = links[i]
        prefix = 'https://www.otodom.pl'
        suffix = url.split('/oferta/')[-1]
        result = prefix + '/oferta/' + suffix
        good_links.append(result)

    # replace df link column with good_links
    df['link'] = good_links
    
    additions = list()
    for x in range(len(df['link'])):
        url = df['link'][x]
        res = requests.get(url)
        soup = BeautifulSoup(res.text, 'html.parser')
        try:
            forma_wlasnosci = soup.find('div', {'aria-label': 'Forma własności'}).find_all('div')[3].text
        except:
            forma_wlasnosci = None
            
        try:
            stan_wykonczenia = soup.find('div', {'aria-label': 'Stan wykończenia'}).find_all('div')[3].text
        except:
            stan_wykonczenia = None
            
        try:
            pietro = soup.find('div', {'aria-label': 'Piętro'}).find_all('div')[3].text
        except:
            pietro = None
            
        try:
            balkon = soup.find('div', {'aria-label': 'Balkon / ogród / taras'}).find_all('div')[3].text
        except:
            balkon = None
            
        try:
            czynsz = soup.find('div', {'aria-label': 'Czynsz'}).find_all('div')[3].text
        except:
            czynsz = None
        
        try:
            parking = soup.find('div', {'aria-label': 'Miejsce parkingowe'}).find_all('div')[3].text
        except:
            parking = None
            
        try:
            ogrzewanie = soup.find('div', {'aria-label': 'Ogrzewanie'}).find_all('div')[3].text
        except:
            ogrzewanie = None
        
        additions.append({
        'forma_wlasnosci': forma_wlasnosci,
        'stan_wykonczenia': stan_wykonczenia,
        'pietro': pietro,
        'balkon': balkon,
        'czynsz': czynsz,
        'parking': parking,
        'ogrzewanie': ogrzewanie
        })
    df = df.join(pd.DataFrame(additions))
    return df

In [35]:
# check for duplicates
otodom_main.drop_duplicates(subset=['name'], keep='first', inplace=True)

In [36]:
otodom_main

,name,city,price,rooms,area,price_per_m2,link,forma_wlasnosci,stan_wykonczenia,pietro,balkon,czynsz,parking,ogrzewanie
0,"Mieszkanie 3 pok. z tarasem, balkon Skarbka z Gór",Warszawa,660 000 zł,3 pokoje,69.9 m²,9442 zł/m²,https://www.otodom.pl/oferta/mieszkanie-3-pok-...,pełna własność,do zamieszkania,parter/2,"balkon, taras",None,garaż/miejsce parkingowe,None
1,3 pokoje Jelonki dwustronne gotowe do zamieszk...,Warszawa,780 000 zł,3 pokoje,60 m²,13 000 zł/m²,https://www.otodom.pl/oferta/3-pokoje-jelonki-...,spółdzielcze własnościowe,do zamieszkania,3/12,balkon,570 zł,None,miejskie
2,"Ciekawe, dwupokojowe mieszkanie na MDM-ie.",Warszawa,1 495 000 zł,2 pokoje,65 m²,23 000 zł/m²,https://www.otodom.pl/oferta/ciekawe-dwupokojo...,pełna własność,do zamieszkania,3,balkon,702 zł,None,miejskie
3,4-pokojowe mieszkanie 80m2 + 2 loggie,Warszawa,Zapytaj o cenę,4 pokoje,80.12 m²,,https://www.otodom.pl/oferta/4-pokojowe-mieszk...,pełna własność,do wykończenia,1/3,None,None,garaż/miejsce parkingowe,None
4,3-pokojowe mieszkanie 59m2 + 2 ogródki,Warszawa,861 204 zł,3 pokoje,59.23 m²,14 540 zł/m²,https://www.otodom.pl/oferta/3-pokojowe-mieszk...,pełna własność,do wykończenia,parter/8,"taras, ogródek",None,garaż/miejsce parkingowe,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,"Komfortowe, 2 pokoje, Ursus",Warszawa,735 000 zł,2 pokoje,56.6 m²,12 986 zł/m²,https://www.otodom.pl/oferta/komfortowe-2-poko...,None,None,5/6,balkon,620 zł,garaż/miejsce parkingowe,miejskie
71,2 pokoje Stara Ochota z ogromnym tarasem na da...,Warszawa,552 000 zł,2 pokoje,40 m²,13 800 zł/m²,https://www.otodom.pl/oferta/2-pokoje-stara-oc...,None,do zamieszkania,1/5,"balkon, taras",400 zł,None,miejskie
72,Mieszkanie z duszą w sercu miasta | 300m do metra,Warszawa,1 020 000 zł,2 pokoje,50 m²,20 400 zł/m²,https://www.otodom.pl/oferta/mieszkanie-z-dusz...,pełna własność,do wykończenia,3/5,None,900 zł,None,miejskie
73,Wyjątkowo cicha lokalizacja | 8 minut Metro Mł...,Warszawa,560 000 zł,2 pokoje,38 m²,14 737 zł/m²,https://www.otodom.pl/oferta/wyjatkowo-cicha-l...,pełna własność,do zamieszkania,2/3,balkon,None,None,miejskie


In [128]:
# Get the HTML from the page
url = 'https://www.otodom.pl'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'html.parser')
# search input
key = 'Warszawa'
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get(url)
time.sleep(2)
#cookie click
cookie_click = driver.find_element(By.XPATH, '//*[@id="onetrust-accept-btn-handler"]')
cookie_click.click()
time.sleep(2)
#search bar click
search_bar = driver.find_element(By.XPATH, '//*[@id="location"]/div[2]')
search_bar.click()
time.sleep(2)
#search entry
search_entry = driver.find_element(By.XPATH, '//*[@id="location-picker-input"]')
search_entry.send_keys(key)
time.sleep(2)
#search select
search_select = driver.find_element(By.XPATH, '//*[@id="__next"]/main/section/div/div/form/div/div[1]/div[3]/div/div[1]/div/div[2]/ul/li[1]/label[1]')
search_select.click()
time.sleep(2)
#search button press
search_button = driver.find_element(By.XPATH, '//*[@id="search-form-submit"]')
search_button.click()
time.sleep(5)
# getting apartments data
# add to url '&limit=72&page=1'
url = driver.current_url
res = requests.get(url)
time.sleep(5)
# scroll to bottom
driver.execute_script("window.scrollTo(10, document.body.scrollHeight);")
button = driver.find_element(By.XPATH, '//*[@id="__next"]/div[2]/main/div/div[2]/div[1]/div[4]/div/div/div/div/div/div[2]/div')
button.click()
time.sleep(3)
# change the number of listings to be shown
pages = driver.find_element(By.XPATH, '//*[@id="react-select-2-option-3"]')
pages.click()
time.sleep(5)
# scroll to bottom
driver.execute_script("window.scrollTo(10, document.body.scrollHeight);")
time.sleep(2)
soup = BeautifulSoup(driver.page_source, 'html.parser')
apartment = soup.find_all('li', {'class': 'css-iq9jxc e1n6ljqa1'})
apt: element.Tag
apartments = list()
for apt in apartment:
    try:
        apartments.append({
            'name': apt.find('h3', {'class': 'css-1mmyqk7 e1n6ljqa6'}).text,
            'city': key,
            'price': apt.find('span', {'class': 'css-1ntk0hg ei6hyam1'}).text,
            'rooms': apt.find('div', {'class': 'e1n6ljqa19 css-6vtodn ei6hyam0'}).find_all('span')[2].get_text(),
            'area': (apt.find('div', {'class': 'e1n6ljqa19 css-6vtodn ei6hyam0'}).find_all('span')[3].get_text()),
            'price_per_m2': apt.find('div', {'class': 'e1n6ljqa19 css-6vtodn ei6hyam0'}).find_all('span')[1].get_text(),
            'link': url + apt.find('a', {'class': 'css-1up0y1q e1n6ljqa16'}).get('href')
        })
    except:
        pass
df = pd.DataFrame(apartments)
fun(df)

,name,city,price,rooms,area,price_per_m2,link,forma_wlasnosci,stan_wykonczenia,pietro,balkon,czynsz,parking,ogrzewanie
0,"NOWE Mieszkanie, 3 pokoje, 58 m2, Wysoki Standard",Warszawa,865 000 zł,3 pokoje,58.36 m²,14 822 zł/m²,https://www.otodom.pl/oferta/nowe-mieszkanie-3...,pełna własność,do zamieszkania,1/5,balkon,600 zł,garaż/miejsce parkingowe,miejskie
1,Właściciel. 49m2. Bliska Wola. Centrum 10 min,Warszawa,1 384 200 zł,2 pokoje,49.49 m²,27 969 zł/m²,https://www.otodom.pl/oferta/wlasciciel-49m2-b...,pełna własność,do zamieszkania,8/20,None,None,None,miejskie
2,"BLISKA WOLA TOWER sprzedam/wynajmę. FV23%,",Warszawa,590 000 zł,2 pokoje,36.43 m²,16 195 zł/m²,https://www.otodom.pl/oferta/bliska-wola-tower...,pełna własność,do wykończenia,> 10/20,balkon,None,garaż/miejsce parkingowe,miejskie
3,2 pokoje Stara Ochota z ogromnym tarasem na da...,Warszawa,552 000 zł,2 pokoje,40 m²,13 800 zł/m²,https://www.otodom.pl/oferta/2-pokoje-stara-oc...,None,do zamieszkania,1/5,"balkon, taras",400 zł,None,miejskie
4,Mieszkanie z duszą w sercu miasta | 300m do metra,Warszawa,1 020 000 zł,2 pokoje,50 m²,20 400 zł/m²,https://www.otodom.pl/oferta/mieszkanie-z-dusz...,pełna własność,do wykończenia,3/5,None,900 zł,None,miejskie
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,2 pokojowe mieszkanie z pięknym widokiem!,Warszawa,525 000 zł,2 pokoje,38 m²,13 816 zł/m²,https://www.otodom.pl/oferta/2-pokojowe-mieszk...,spółdzielcze wł. z KW,do zamieszkania,6/12,balkon,700 zł,garaż/miejsce parkingowe,miejskie
71,wygodne mieszkanie 4-pokojowe Białostocka,Warszawa,860 000 zł,4 pokoje,76.7 m²,11 213 zł/m²,https://www.otodom.pl/oferta/wygodne-mieszkani...,None,None,8/10,balkon,800 zł,None,miejskie
72,Rozkładowe - 49 m²| Wawrzyszew | Metro | Loggia,Warszawa,629 000 zł,2 pokoje,49 m²,12 837 zł/m²,https://www.otodom.pl/oferta/rozkladowe-49-m-w...,None,do zamieszkania,7/11,None,400 zł,garaż/miejsce parkingowe,miejskie
73,2 pok. * 40 m2 * Obok metra M1-M2 * Od zaraz!,Warszawa,620 000 zł,2 pokoje,40.3 m²,15 385 zł/m²,https://www.otodom.pl/oferta/2-pok-40-m2-obok-...,pełna własność,do wykończenia,8/15,balkon,400 zł,garaż/miejsce parkingowe,miejskie
